In [1]:
from cffi import FFI

In [2]:
ffi = FFI()
ffi.cdef("""
    long __stdcall IV_open();
    long __stdcall IV_close();
    long __stdcall IV_MaxDevices();
    long __stdcall IV_selectdevice(long *devnr);
    long __stdcall IV_getdevicestatus();
    long __stdcall IV_readSN(char *sntext);
    long __stdcall IV_connect(long* devconnect);
    long __stdcall IV_VersionHost(long *version);
    long __stdcall IV_VersionDll();
    long __stdcall IV_VersionCheck();
    long __stdcall IV_HostHandle();
    long __stdcall IV_VersionDllFile();
    long __stdcall IV_VersionDllFileStr();
    long __stdcall IV_SelectChannel(long *channel);
    
    long __stdcall IV_getcellstatus(long *devcellstatus);
    long __stdcall IV_setconnectionmode(long *value);
    long __stdcall IV_setcellon(long *cellon);
    long __stdcall IV_setpotential(double *value);
    long __stdcall IV_setpotentialWE2(double *value);
    long __stdcall IV_setcurrent(double *value);
    long __stdcall IV_getpotential(double *value);
    long __stdcall IV_setcurrentrange(long *value);
    long __stdcall IV_setcurrentrangeWE2(long *value);
    long __stdcall IV_getcurrent(double *value);
    long __stdcall IV_getcurrentWE2(double *value);
    long __stdcall IV_setfilter(long *value);
    long __stdcall IV_setstability(long *value);
    long __stdcall IV_setbistatmode(long *value);
    long __stdcall IV_setdac(long *channr, double *value);
    long __stdcall IV_getadc(long *channr, double *value);
    long __stdcall IV_setmuxchannel(long *value);
    long __stdcall IV_setdigout(long *value);
    long __stdcall IV_getdigin(long *value);
    long __stdcall IV_setfrequency(double *value);
    long __stdcall IV_setamplitude(double *value);
    long __stdcall IV_getcurrenttrace(long* npoints, double *rate, double *values);
    long __stdcall IV_getcurrentWE2trace(long* npoints, double *rate, double *values);
    long __stdcall IV_getpotentialtrace(long* npoints, double *rate, double *values);

    long __stdcall IV_we32setchannel(long *index);
    long __stdcall IV_we32setoffset(long *index, double *value);
    long __stdcall IV_we32setoffsets(long *nval, double *values);
    long __stdcall IV_we32getoffsets(long *nval, double *values);
    long __stdcall IV_we32readcurrents(double *values);

    long __stdcall IV_readmethod(char *fname);
    long __stdcall IV_savemethod(char *fname);
    long __stdcall IV_startmethod(char *fname);
    long __stdcall IV_abort();
    long __stdcall IV_savedata(char *fname);
    long __stdcall IV_setmethodparameter(char *parname, char *parvalue);
    long __stdcall IV_Ndatapoints(long *value);
    long __stdcall IV_getdata(long *pointnr, double *x, double *y, double *z);
    long __stdcall IV_getdatafromline(long *pointnr, long *scannr, double *x, double *y, double *z);

    long __stdcall IV_getDbFileName(char *fname);

    long __stdcall IV_StatusParGet(long *value);
    long __stdcall IV_StatusParSet(long *value);
""")

## Import dll

In [3]:
lib = ffi.dlopen("../pyvium_core/IVIUM_remdriver64.dll")

## IV_open()
###### no Iviumsoft --> -1
###### success --> 0
###### already open --> 0

In [4]:
def open_driver():
    return lib.IV_open()

In [5]:
# iv_open()

## IV_close()

In [6]:
def close_driver():
    return lib.IV_close()

In [7]:
# iv_close()
# iv_open()

## IV_MaxDevices()

In [8]:
def get_max_device_number():
    return lib.IV_MaxDevices()

In [9]:
get_max_device_number()

24

## IV_readSN(char *sntext)
######  no IV_open() --> crash
######  no Iviumsoft --> -1 (rc=-1)
######  no device --> '' (rc=0)
######  device --> serialNum (rc=0)

In [10]:
def read_serial_number():
    serial_number_char_16 = ffi.new("char[]", 16)
    rc = lib.IV_readSN(serial_number_char_16)
   
    return rc, ffi.string(serial_number_char_16).decode("utf-8")

# iv_readSN()

## IV_selectdevice(long *devnr)
######  no IV_open() --> crash
######  IV_open() --> integer values


In [11]:
def select_iviumsoft_instance(iviumsoft_instance_number):
    instance_number = ffi.new("long *", iviumsoft_instance_number)
    rc = lib.IV_selectdevice(instance_number)

    return rc

# iv_close()
# iv_open()
# print(iv_select_device(1))
# print(2, iv_selectdevice(1))

## IV_connect(long *devconnect)
######  no IV_open() --> crash
######  IV_open() --> with 1 connects device and with 0 disconnects it
###### software not running => -1
###### no device connected through usb => 1
###### device connected through usb    => 0 (device connected in iviumsoft)


In [12]:
## connection_status: 1 connect, 0 disconnect
def connect_device():
    connection_on = ffi.new("long *", 1)
    return lib.IV_connect(connection_on)

def disconnect_device():
    connection_off = ffi.new("long *", 0)
    return lib.IV_connect(connection_off)

open_driver()
# print(iv_select_iviumsoft_instance(1))
print(connect_device())
# print(2, iv_disconnect_device())

0


## IV_VersionDll()
######  no IV_open() --> crash
######  IV_open() --> returns the version for the driver's dll without points (200)


In [13]:
open_driver()
def get_dll_version():
    return lib.IV_VersionDll()

# print(iv_get_dll_version())

## IV_VersionCheck()
######  no IV_open() --> crash
######  IV_open() --> returns 1 if the ivium software is running, -255 otherwise

In [14]:
open_driver()
def is_iviumsoft_running():
    if lib.IV_VersionCheck() == 1:
        return True
    else:
        return False

print(is_iviumsoft_running())

True



## IV_getdevicestatus()

-1=no IviumSoft; 0=not connected; 1=available_idle; 2=available_busy; 3=no device available


In [ ]:
def get_device_status():
    rc = lib.IV_getdevicestatus()
    print(rc)
    return lib.IV_getdevicestatus()
    
get_device_status()